# Download Data

# Import

In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.signal import correlate
from scipy.signal import periodogram
from scipy.signal import welch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from statsmodels.tsa.ar_model import AutoReg
from scipy.linalg import toeplitz

# Load Data

In [2]:
data = loadmat("Project_data.mat")

# Definition

### Variables

In [3]:
#train test data
X_train = data["TrainData"]
X_test = data["TestData"]
y_train = data["TrainLabels"]

#classification
class1 = X_train[:,:,y_train.T[:,0] == 1]
class2 = X_train[:,:,y_train.T[:,0] == -1]

#normalizing domain of signals
scaler = StandardScaler()
normalizedX_train = []
normalizedClass1 = []
normalizedClass2 = []

for i in range(59):
  a = scaler.fit_transform(X_train[i,:,:].T)
  b = scaler.fit_transform(class1[i,:,:].T)
  c = scaler.fit_transform(class2[i,:,:].T)
  normalizedX_train.append(np.array(a).T)
  normalizedClass1.append(np.array(b).T)
  normalizedClass2.append(np.array(c).T)

normalizedX_train=np.array(normalizedX_train)
normalizedClass1=np.array(normalizedClass1)
normalizedClass2=np.array(normalizedClass2)

In [4]:
X_train.shape ,X_test.shape , y_train.shape , class1.shape , class2.shape , normalizedX_train.shape , normalizedClass1.shape , normalizedClass2.shape

((59, 5000, 550),
 (59, 5000, 159),
 (1, 550),
 (59, 5000, 277),
 (59, 5000, 273),
 (59, 5000, 550),
 (59, 5000, 277),
 (59, 5000, 273))

### Functions

In [5]:
def calculate_form_factor(signal):
    f_d = np.diff(signal)
    s_d = np.diff(f_d)
    signal_std = np.std(signal)
    
    return (signal_std * np.std(s_d))/ (np.std(f_d)**2)

In [6]:
def fisher1D(mu1,mu2,var1,var2,mu):
  return ( ((mu1-mu)**2) + ((mu2 - mu) **2) ) /(var1 + var2)

In [7]:
def calculate_ar1(signal , samples , order):
    acf = np.correlate(signal, signal, mode='full')
    acf_positive = acf[samples - 1:samples + order]
    return np.linalg.solve(toeplitz(acf_positive), acf_positive)



def calculate_ar2(signal, order):
    model = AutoReg( X_train[0,:,0], lags=order)
    model_fit = model.fit()
    return np.array(model_fit.params)

In [8]:
def normalize(feature):
    scaler = MinMaxScaler()
    normalized_feature = scaler.fit_transform(feature)
    return normalized_feature

# Implementation

In [9]:
fishersOfFeatures=[]
features_count=np.zeros((550,1))

In [10]:
#variance
vars = np.var(X_train, axis= 1)
varsClass1= np.var(class1, axis= 1)
varsClass2= np.var(class2, axis= 1)

for i in range(59):
  ithVars=vars[i,:]  
  ithVarsClass1=varsClass1[i,:]
  ithVarsClass2=varsClass2[i,:]  

  features_count=np.concatenate((features_count,ithVars[:,np.newaxis]),axis=1)
    
  mu=ithVars.mean()
  mu1=ithVarsClass1.mean()
  mu2=ithVarsClass2.mean()
  var1=ithVarsClass1.var()  
  var2=ithVarsClass2.var()
  fishersOfFeatures.append(fisher1D(mu1,mu2,var1,var2,mu))

In [11]:
features_count.shape

(550, 60)

In [12]:
#histogram
for i in range(59):
    hists=[]
    hists1=[]
    hists2=[]

    for j in range(X_train.shape[2]):
      hists.append(np.histogram(X_train[i,:,j],bins=10)[0])
    for j in range(class1.shape[2]):
      hists1.append(np.histogram(class1[i,:,j],bins=10)[0])
    for j in range(class2.shape[2]):
      hists2.append(np.histogram(class2[i,:,j],bins=10)[0])

    hists=np.array(hists)
    hists1=np.array(hists1)
    hists2=np.array(hists2)
    
    

    for bin in range(10):
        
        features_count=np.concatenate((features_count,hists[:,bin][:,np.newaxis]),axis=1)
        
        mu=hists[:,bin].mean()
        mu1=hists1[:,bin].mean()
        var1=hists1[:,bin].var()
        mu2=hists2[:,bin].mean()
        var2=hists2[:,bin].var()
        fishersOfFeatures.append(fisher1D(mu1,mu2,var1,var2,mu))

In [13]:
features_count.shape

(550, 650)

In [14]:
#form factor
for i in range(59):
  ffs=[]
  ffs1=[]
  ffs2=[]

  for j in range(X_train.shape[2]):
    ffs.append(calculate_form_factor(X_train[i,:,j]))

  for j in range(class1.shape[2]):
    ffs1.append(calculate_form_factor(class1[i,:,j]))

  for j in range(class2.shape[2]):
    ffs2.append(calculate_form_factor(class2[i,:,j]))

  ffs=np.array(ffs)
  ffs1=np.array(ffs1)
  ffs2=np.array(ffs2)

  features_count=np.concatenate((features_count,ffs[:,np.newaxis]),axis=1)  

  mu=ffs.mean()  
  mu1=ffs1.mean()
  mu2=ffs2.mean()
  var1=ffs1.var()
  var2=ffs2.var()
  #print(mu,mu1,mu2,var1,var2)  
  fishersOfFeatures.append(fisher1D(mu1,mu2,var1,var2,mu))

In [15]:
features_count.shape

(550, 709)

In [16]:
#AR
# fishersOfFeatures=[]
# for i in range(59):
#     ars=[]
#     ars1=[]
#     ars2=[]

#     for j in range(X_train.shape[2]):
#       ars.append(calculate_ar1(X_train[i,:,j],5000,10))
#     for j in range(class1.shape[2]):
#       ars1.append(calculate_ar1(class1[i,:,j],5000,10))
#     for j in range(class2.shape[2]):
#       ars2.append(calculate_ar1(class2[i,:,j],5000,10))

#     ars=np.array(ars)
#     ars1=np.array(ars1)
#     ars2=np.array(ars2)
    
#     for bin in range(1,11):
#         mu=ars[:,bin].mean()
#         mu1=ars1[:,bin].mean()
#         var1=ars1[:,bin].var()
#         mu2=ars2[:,bin].mean()
#         var2=ars2[:,bin].var()
#         fishersOfFeatures.append(fisher1D(mu1,mu2,var1,var2,mu))

In [17]:
#fishersOfFeatures

In [18]:
#correlation
corr=np.zeros((class1.shape[2],59,59))
for i in range(59):
  for j in range(59):
    if i>j:
      corrs=[]
      corrs1=[]
      corrs2=[]
      
        
      for k in range(X_train.shape[2]):
        corrs.append(np.corrcoef(X_train[i,:,k],X_train[j,:,k]))
      for k in range(class1.shape[2]):
        corrs1.append(np.corrcoef(class1[i,:,k],class1[j,:,k]))
      for k in range(class2.shape[2]):
        corrs2.append(np.corrcoef(class2[i,:,k],class2[j,:,k]))

      corrs=np.array(corrs)
      corrs1=np.array(corrs1)
      corrs2=np.array(corrs2) 
      
      features_count=np.concatenate((features_count,corrs[:,0,1][:,np.newaxis]),axis=1)    

      mu=corrs.mean()
      mu1=corrs1.mean()
      mu2=corrs2.mean()
      var1=corrs1.var()
      var2=corrs2.var()
      fishersOfFeatures.append(fisher1D(mu1,mu2,var1,var2,mu))

In [19]:
features_count.shape

(550, 2420)

In [20]:
#maxfreq
fs=1000
for i in range(59):
      
    Fs=[]
    Fs1=[]
    Fs2=[]
    
    for j in range(X_train.shape[2]):
        (f,S) = periodogram(X_train[i,:,j] , fs , scaling="density")
        Fs.append(f[np.argmax(S)])
        
    for j in range(class1.shape[2]):
        (f,S) = periodogram(class1[i,:,j] , fs , scaling="density")
        Fs1.append(f[np.argmax(S)])
    
    for j in range(class2.shape[2]):
        (f,S) = periodogram(class2[i,:,j] , fs , scaling="density")
        Fs2.append(f[np.argmax(S)])
        
    Fs=np.array(Fs)
    Fs1=np.array(Fs1)
    Fs2=np.array(Fs2)
        
    features_count=np.concatenate((features_count,Fs[:,np.newaxis]),axis=1)
    
    mu=Fs.mean()
    mu1=Fs1.mean()
    mu2=Fs2.mean()
    var1=Fs1.var()
    var2=Fs2.var()
    fishersOfFeatures.append(fisher1D(mu1,mu2,var1,var2,mu))

In [21]:
features_count.shape

(550, 2479)

In [22]:
#meanfreq
fs=1000

for i in range(59):
      
    Fs=[]
    Fs1=[]
    Fs2=[]
    
    for j in range(X_train.shape[2]):
        (f,S) = periodogram(X_train[i,:,j] , fs , scaling="density")
        Fs.append(sum(f*S)/sum(S))
        
    for j in range(class1.shape[2]):
        (f,S) = periodogram(class1[i,:,j] , fs , scaling="density")
        Fs1.append(sum(f*S)/sum(S))
    
    for j in range(class2.shape[2]):
        (f,S) = periodogram(class2[i,:,j] , fs , scaling="density")
        Fs2.append(sum(f*S)/sum(S))
        
    Fs=np.array(Fs)
    Fs1=np.array(Fs1)
    Fs2=np.array(Fs2)
    
    features_count=np.concatenate((features_count,Fs[:,np.newaxis]),axis=1)
        
    mu=Fs.mean()
    mu1=Fs1.mean()
    mu2=Fs2.mean()
    var1=Fs1.var()
    var2=Fs2.var()
    fishersOfFeatures.append(fisher1D(mu1,mu2,var1,var2,mu))

In [23]:
features_count.shape

(550, 2538)

In [24]:
#medfreq
fs=1000

for i in range(59):
      
    Fs=[]
    Fs1=[]
    Fs2=[]
    
    for j in range(X_train.shape[2]):
        (f,S) = periodogram(X_train[i,:,j] , fs , scaling="density")
        Fs.append( f[ np.where( np.cumsum(S) >= np.cumsum(S)[-1]/2 )[0][0] ] )
        
    for j in range(class1.shape[2]):
        (f,S) = periodogram(class1[i,:,j] , fs , scaling="density")
        Fs1.append( f[ np.where( np.cumsum(S) >= np.cumsum(S)[-1]/2 )[0][0] ] )
    
    for j in range(class2.shape[2]):
        (f,S) = periodogram(class2[i,:,j] , fs , scaling="density")
        Fs2.append( f[ np.where( np.cumsum(S) >= np.cumsum(S)[-1]/2 )[0][0] ] )
        
    Fs=np.array(Fs)
    Fs1=np.array(Fs1)
    Fs2=np.array(Fs2)
        
    features_count=np.concatenate((features_count,Fs[:,np.newaxis]),axis=1)
    
    mu=Fs.mean()
    mu1=Fs1.mean()
    mu2=Fs2.mean()
    var1=Fs1.var()
    var2=Fs2.var()
    fishersOfFeatures.append(fisher1D(mu1,mu2,var1,var2,mu))

In [25]:
features_count.shape

(550, 2597)

In [26]:
#bandpower
fs=1000

for i in range(59):
    
        delta=[0.1,3]
        eRelDeltas=[]
        eRelDeltas1=[]
        eRelDeltas2=[]
        
        theta=[4,7]
        eRelThetas=[]
        eRelThetas1=[]
        eRelThetas2=[]
        
        alpha=[8,12]
        eRelAlphas=[]
        eRelAlphas1=[]
        eRelAlphas2=[]
        
        lowRangeBeta=[12,15]
        eRelLowRangeBetas=[]
        eRelLowRangeBetas1=[]
        eRelLowRangeBetas2=[]

        midRangeBeta=[16,20]
        eRelMidRangeBetas=[]
        eRelMidRangeBetas1=[]
        eRelMidRangeBetas2=[]
        
        highRangeBeta=[21,30]
        eRelHighRangeBetas=[]
        eRelHighRangeBetas1=[]
        eRelHighRangeBetas2=[]
        
        gamma=[30,100]
        eRelGammas=[]
        eRelGammas1=[]
        eRelGammas2=[]
        

        for j in range(X_train.shape[2]):
            
            (f,S) = periodogram(X_train[i,:,j] , fs , scaling="density")
            
            eDelta=sum(S[ (f>=delta[0]) & (f<=delta[1])]**2)
            eTheta=sum(S[ (f>=theta[0]) & (f<=theta[1])]**2)
            eAlpha=sum(S[(f>=alpha[0]) & (f<=alpha[1])]**2)
            eLowRangeBeta=sum(S[ (f>=lowRangeBeta[0]) & (f<=lowRangeBeta[1])]**2)
            eMidRangeBeta=sum(S[ (f>=midRangeBeta[0]) & (f<=midRangeBeta[1])]**2)
            eHighRangeBeta=sum(S[ (f>=highRangeBeta[0]) & (f<=highRangeBeta[1])]**2)
            eGamma=sum(S[ (f>=gamma[0]) & (f<=gamma[1])]**2)
            e=eDelta+eTheta+eAlpha+eLowRangeBeta+eMidRangeBeta+eHighRangeBeta+eGamma
            
            eRelDeltas.append(eDelta/e)
            eRelThetas.append(eTheta/e)
            eRelAlphas.append(eAlpha/e)
            eRelLowRangeBetas.append(eLowRangeBeta/e)
            eRelMidRangeBetas.append(eMidRangeBeta/e)
            eRelHighRangeBetas.append(eHighRangeBeta/e)
            eRelGammas.append(eGamma/e)
            

        for j in range(class1.shape[2]):
            (f,S) = periodogram(class1[i,:,j] , fs , scaling="density")
            eDelta=sum(S[ (f>=delta[0]) & (f<=delta[1])]**2)
            eTheta=sum(S[ (f>=theta[0]) & (f<=theta[1])]**2)
            eAlpha=sum(S[(f>=alpha[0]) & (f<=alpha[1])]**2)
            eLowRangeBeta=sum(S[ (f>=lowRangeBeta[0]) & (f<=lowRangeBeta[1])]**2)
            eMidRangeBeta=sum(S[ (f>=midRangeBeta[0]) & (f<=midRangeBeta[1])]**2)
            eHighRangeBeta=sum(S[ (f>=highRangeBeta[0]) & (f<=highRangeBeta[1])]**2)
            eGamma=sum(S[ (f>=gamma[0]) & (f<=gamma[1])]**2)
            e=eDelta+eTheta+eAlpha+eLowRangeBeta+eMidRangeBeta+eHighRangeBeta+eGamma
            
            eRelDeltas1.append(eDelta/e)
            eRelThetas1.append(eTheta/e)
            eRelAlphas1.append(eAlpha/e)
            eRelLowRangeBetas1.append(eLowRangeBeta/e)
            eRelMidRangeBetas1.append(eMidRangeBeta/e)
            eRelHighRangeBetas1.append(eHighRangeBeta/e)
            eRelGammas1.append(eGamma/e)

        for j in range(class2.shape[2]):
            (f,S) = periodogram(class2[i,:,j] , fs , scaling="density")
            eDelta=sum(S[ (f>=delta[0]) & (f<=delta[1])]**2)
            eTheta=sum(S[ (f>=theta[0]) & (f<=theta[1])]**2)
            eAlpha=sum(S[(f>=alpha[0]) & (f<=alpha[1])]**2)
            eLowRangeBeta=sum(S[ (f>=lowRangeBeta[0]) & (f<=lowRangeBeta[1])]**2)
            eMidRangeBeta=sum(S[ (f>=midRangeBeta[0]) & (f<=midRangeBeta[1])]**2)
            eHighRangeBeta=sum(S[ (f>=highRangeBeta[0]) & (f<=highRangeBeta[1])]**2)
            eGamma=sum(S[ (f>=gamma[0]) & (f<=gamma[1])]**2)
            e=eDelta+eTheta+eAlpha+eLowRangeBeta+eMidRangeBeta+eHighRangeBeta+eGamma
            
            eRelDeltas2.append(eDelta/e)
            eRelThetas2.append(eTheta/e)
            eRelAlphas2.append(eAlpha/e)
            eRelLowRangeBetas2.append(eLowRangeBeta/e)
            eRelMidRangeBetas2.append(eMidRangeBeta/e)
            eRelHighRangeBetas2.append(eHighRangeBeta/e)
            eRelGammas2.append(eGamma/e)

        eRelDeltas=np.array(eRelDeltas)
        eRelThetas=np.array(eRelThetas)
        eRelAlphas=np.array(eRelAlphas)
        eRelLowRangeBetas=np.array(eRelLowRangeBetas)
        eRelMidRangeBetas=np.array(eRelMidRangeBetas)
        eRelHighRangeBetas=np.array(eRelHighRangeBetas)
        eRelGammas=np.array(eRelGammas)

        eRelDeltas1=np.array(eRelDeltas1)
        eRelThetas1=np.array(eRelThetas1)
        eRelAlphas1=np.array(eRelAlphas1)
        eRelLowRangeBetas1=np.array(eRelLowRangeBetas1)
        eRelMidRangeBetas1=np.array(eRelMidRangeBetas1)
        eRelHighRangeBetas1=np.array(eRelHighRangeBetas1)
        eRelGammas1=np.array(eRelGammas1)

        eRelDeltas2=np.array(eRelDeltas2)
        eRelThetas2=np.array(eRelThetas2)
        eRelAlphas2=np.array(eRelAlphas2)
        eRelLowRangeBetas2=np.array(eRelLowRangeBetas2)
        eRelMidRangeBetas2=np.array(eRelMidRangeBetas2)
        eRelHighRangeBetas2=np.array(eRelHighRangeBetas2)
        eRelGammas2=np.array(eRelGammas2)

        features_count=np.concatenate((features_count,eRelDeltas[:,np.newaxis]),axis=1)
        features_count=np.concatenate((features_count,eRelThetas[:,np.newaxis]),axis=1)
        features_count=np.concatenate((features_count,eRelAlphas[:,np.newaxis]),axis=1)
        features_count=np.concatenate((features_count,eRelLowRangeBetas[:,np.newaxis]),axis=1)
        features_count=np.concatenate((features_count,eRelMidRangeBetas[:,np.newaxis]),axis=1)
        features_count=np.concatenate((features_count,eRelHighRangeBetas[:,np.newaxis]),axis=1)
        features_count=np.concatenate((features_count,eRelGammas[:,np.newaxis]),axis=1)

        muDelta=eRelDeltas.mean()
        muDelta1=eRelDeltas1.mean()
        muDelta2=eRelDeltas2.mean()
        varDelta1=eRelDeltas1.var()
        varDelta2=eRelDeltas2.var()
        fishersOfFeatures.append(fisher1D(muDelta1,muDelta2,varDelta1,varDelta2,muDelta))
        
        
        muTheta=eRelThetas.mean()
        muTheta1=eRelThetas1.mean()
        muTheta2=eRelThetas2.mean()
        varTheta1=eRelThetas1.var()
        varTheta2=eRelThetas2.var()
        fishersOfFeatures.append(fisher1D(muTheta1,muTheta2,varTheta1,varTheta2,muTheta))

        
        muAlpha=eRelAlphas.mean()
        muAlpha1=eRelAlphas1.mean()
        muAlpha2=eRelAlphas2.mean()
        varAlpha1=eRelAlphas1.var()
        varAlpha2=eRelAlphas2.var()
        fishersOfFeatures.append(fisher1D(muAlpha1,muAlpha2,varAlpha1,varAlpha2,muAlpha))

        
        muLowRangeBeta=eRelLowRangeBetas.mean()
        muLowRangeBeta1=eRelLowRangeBetas1.mean()
        muLowRangeBeta2=eRelLowRangeBetas2.mean()
        varLowRangeBeta1=eRelLowRangeBetas1.var()
        varLowRangeBeta2=eRelLowRangeBetas2.var()
        fishersOfFeatures.append(fisher1D(muLowRangeBeta1,muLowRangeBeta2,varLowRangeBeta1,varLowRangeBeta2,muLowRangeBeta))

        
        muMidRangeBeta=eRelMidRangeBetas.mean()
        muMidRangeBeta1=eRelMidRangeBetas1.mean()
        muMidRangeBeta2=eRelMidRangeBetas2.mean()
        varMidRangeBeta1=eRelMidRangeBetas1.var()
        varMidRangeBeta2=eRelMidRangeBetas2.var()
        fishersOfFeatures.append(fisher1D(muMidRangeBeta1,muMidRangeBeta2,varMidRangeBeta1,varMidRangeBeta2,muMidRangeBeta))

        
        muHighRangeBeta=eRelHighRangeBetas.mean()
        muHighRangeBeta1=eRelHighRangeBetas1.mean()
        muHighRangeBeta2=eRelHighRangeBetas2.mean()
        varHighRangeBeta1=eRelHighRangeBetas1.var()
        varHighRangeBeta2=eRelHighRangeBetas2.var()
        fishersOfFeatures.append(fisher1D(muHighRangeBeta1,muHighRangeBeta2,varHighRangeBeta1,varHighRangeBeta2,muHighRangeBeta))

        
        muGamma=eRelGammas.mean()
        muGamma1=eRelGammas1.mean()
        muGamma2=eRelGammas2.mean()
        varGamma1=eRelGammas1.var()
        varGamma2=eRelGammas2.var()
        fishersOfFeatures.append(fisher1D(muGamma1,muGamma2,varGamma1,varGamma2,muGamma))


In [27]:
features_count.shape

(550, 3010)

In [28]:
df = pd.DataFrame(fishersOfFeatures, columns=['column_name'])
df.to_csv('fishersOfFeatures.csv', index=False)

In [30]:
fishersOfFeatures=pd.read_csv("fishersOfFeatures.csv")
fishersOfFeatures=fishersOfFeatures.to_numpy().tolist()
sorted_fisher_of_features=fishersOfFeatures.copy()
sorted_fisher_of_features.sort(reverse=True)
best_fishers=sorted_fisher_of_features[0:50]

indexs=[]
for i in range(50):
    indexs.append(fishersOfFeatures.index(best_fishers[i]))

print(indexs)

[2989, 3003, 2940, 161, 2579, 2896, 252, 164, 2500, 542, 2442, 141, 255, 2461, 254, 2954, 2441, 2756, 545, 2890, 2946, 160, 541, 2559, 2996, 2592, 142, 2973, 2569, 2595, 2590, 145, 162, 2510, 2642, 2536, 144, 2895, 2472, 2469, 303, 2583, 2826, 234, 256, 42, 165, 2923, 2533, 2847]


In [31]:
features=[]

for i in range(59):
    features.append("var channel "+str(i+1))
    
for i in range(59):
    for bin in range(10):
        features.append("bin "+str(bin+1)+" channel "+str(i+1))
        
for i in range(59):
    features.append("form factor channel "+str(i+1))
    
for i in range(59):
    for j in range(59):
        if i>j:
            features.append("corr channel "+str(i+1)+" and "+str(j+1))

for i in range(59):
    features.append("maxfreq channel "+str(i+1))

for i in range(59):
    features.append("meanfreq channel "+str(i+1))

for i in range(59):
    features.append("medfreq channel "+str(i+1))
    
for i in range(59):
    features.append("delta channel "+str(i+1))
    features.append("theta channel "+str(i+1))
    features.append("alpha channel "+str(i+1))
    features.append("low range beta channel "+str(i+1))
    features.append("mid range beta channel "+str(i+1))
    features.append("high range beta channel "+str(i+1))
    features.append("gamma channel "+str(i+1))
    
print(features)


['var channel 1', 'var channel 2', 'var channel 3', 'var channel 4', 'var channel 5', 'var channel 6', 'var channel 7', 'var channel 8', 'var channel 9', 'var channel 10', 'var channel 11', 'var channel 12', 'var channel 13', 'var channel 14', 'var channel 15', 'var channel 16', 'var channel 17', 'var channel 18', 'var channel 19', 'var channel 20', 'var channel 21', 'var channel 22', 'var channel 23', 'var channel 24', 'var channel 25', 'var channel 26', 'var channel 27', 'var channel 28', 'var channel 29', 'var channel 30', 'var channel 31', 'var channel 32', 'var channel 33', 'var channel 34', 'var channel 35', 'var channel 36', 'var channel 37', 'var channel 38', 'var channel 39', 'var channel 40', 'var channel 41', 'var channel 42', 'var channel 43', 'var channel 44', 'var channel 45', 'var channel 46', 'var channel 47', 'var channel 48', 'var channel 49', 'var channel 50', 'var channel 51', 'var channel 52', 'var channel 53', 'var channel 54', 'var channel 55', 'var channel 56', 

In [32]:
for i in range(50):
    print(features[indexs[i]])

theta channel 57
theta channel 59
theta channel 50
bin 3 channel 11
medfreq channel 43
gamma channel 43
bin 4 channel 20
bin 6 channel 11
meanfreq channel 23
bin 4 channel 49
maxfreq channel 24
bin 3 channel 9
bin 7 channel 20
maxfreq channel 43
bin 6 channel 20
theta channel 52
maxfreq channel 23
gamma channel 23
bin 7 channel 49
delta channel 43
delta channel 51
bin 2 channel 11
bin 3 channel 49
medfreq channel 23
theta channel 58
medfreq channel 56
bin 4 channel 9
gamma channel 54
medfreq channel 33
medfreq channel 59
medfreq channel 54
bin 7 channel 9
bin 4 channel 11
meanfreq channel 33
mid range beta channel 7
meanfreq channel 59
bin 6 channel 9
high range beta channel 43
maxfreq channel 54
maxfreq channel 51
bin 5 channel 25
medfreq channel 47
gamma channel 33
bin 6 channel 18
bin 8 channel 20
var channel 43
bin 7 channel 11
high range beta channel 47
meanfreq channel 56
gamma channel 36


In [34]:
df = pd.DataFrame(features_count[:,1:], columns=features)
df.to_csv('features_count.csv', index=False)

In [40]:
features_count=pd.read_csv("features_count.csv")
print(features_count.shape)
features_count=features_count.to_numpy()[:,indexs]
print(features_count.shape)

df = pd.DataFrame(features_count, columns=np.array(features)[indexs])
df.to_csv('selcted_features_count.csv', index=False)

(550, 3009)
(550, 50)


In [43]:
selcted_features_count=pd.read_csv("selcted_features_count.csv")
selcted_features_count.to_numpy().shape

(550, 50)